<a href="https://colab.research.google.com/github/pati-dev/text-multi-label/blob/master/attributes_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boot

In [1]:
!pip install scikit-multilearn

%reload_ext tensorboard
%tensorflow_version 2.x
import datetime
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

import gensim

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

     |████████████████████████████████| 92kB 2.9MB/s 
TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
finalDF = pd.read_csv('/content/drive/My Drive/Search Project/finalDF.csv')
finalDF.head()

,business_id,text,RestaurantsReservations,RestaurantsGoodForGroups,OutdoorSeating,RestaurantsTakeOut,GoodForKids,RestaurantsDelivery,BikeParking,Wifi
0,-7BCZH437U5FjmNJ26llkg,this is a nice destination especially for thos...,True,True,False,True,True,True,True,True
1,-9u2uFwhlj3Yq4mG5IweoA,i am not sure what to think of this place hips...,True,False,False,True,False,True,True,False
2,-BbnAc9YEO6pjvJGEtFbVQ,lovely little place for ramen amongst the many...,False,True,False,True,True,False,True,False
3,-CLTZ0cO-ooKDDmXHFoGMg,this was a planned visit as we were visiting a...,False,False,False,True,True,True,True,False
4,-EX_qttyhm5YNb8NWtAcCQ,fun little bar for some country music i would ...,False,True,False,True,False,False,True,False


# Word Clouds

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsReservations
subset = finalDF[finalDF.RestaurantsReservations==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsReservations",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsGoodForGroups
subset = finalDF[finalDF.RestaurantsGoodForGroups==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsGoodForGroups",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# OutdoorSeating
subset = finalDF[finalDF.OutdoorSeating==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("OutdoorSeating",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsTakeOut
subset = finalDF[finalDF.RestaurantsTakeOut==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsTakeOut",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# GoodForKids
subset = finalDF[finalDF.GoodForKids==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("GoodForKids",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsDelivery
subset = finalDF[finalDF.RestaurantsDelivery==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsDelivery",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# BikeParking
subset = finalDF[finalDF.BikeParking==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("BikeParking",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# Wifi
subset = finalDF[finalDF.Wifi==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("Wifi",fontsize=40)
plt.imshow(cloud)

# Data Preparation

In [4]:
train, test = train_test_split(finalDF, test_size=0.30, shuffle=True)
print(train.shape, test.shape)

(1316, 10) (564, 10)


## Method 1: TF-IDF using *sci-kit learn*

In [5]:
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,1), norm='l2')

vectorizer.fit(train.text.values)
vectorizer.fit(test.text.values)

x_train = vectorizer.transform(train.text.values)
y_train = train.drop(labels = ['business_id','text'], axis=1)

x_test = vectorizer.transform(test.text.values)
y_test = test.drop(labels = ['business_id','text'], axis=1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1316, 46631) (1316, 8)
(564, 46631) (564, 8)


## Method 2: Tokenizer using *Keras*

In [11]:
NUM_WORDS = 20000
SEQ_LEN = 5000

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, lower=True)
tokenizer.fit_on_texts(train.text.values)
sequences = tokenizer.texts_to_sequences(train.text.values)
x_train = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=SEQ_LEN)
y_train = train.drop(labels = ['business_id','text'], axis=1)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, lower=True)
tokenizer.fit_on_texts(test.text.values)
sequences = tokenizer.texts_to_sequences(test.text.values)
x_test = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=SEQ_LEN)
y_test = test.drop(labels = ['business_id','text'], axis=1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1316, 5000) (1316, 8)
(564, 5000) (564, 8)


## Method 3: Word2vec using *Gensim*

In [33]:
y_train

,RestaurantsReservations,RestaurantsGoodForGroups,OutdoorSeating,RestaurantsTakeOut,GoodForKids,RestaurantsDelivery,BikeParking,Wifi
260,True,True,False,True,True,False,True,False
1500,True,True,True,True,False,False,True,False
933,True,True,False,True,True,True,True,False
1429,True,True,True,True,True,False,True,True
631,True,True,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...
987,True,True,False,True,True,False,True,True
1011,True,True,True,True,True,False,True,True
1856,True,True,False,True,False,False,True,False
891,True,True,False,True,True,True,True,True


In [20]:
NUM_WORDS = 10000
SEQ_LEN = 999

def word2idx(word):
  return w2v_model.wv.vocab[word].index
def idx2word(idx):
  return w2v_model.wv.index2word[idx]

# Train data
sentences = [[word for word in txt.lower().split()[:NUM_WORDS]] for txt in train.text.values]
w2v_model = gensim.models.Word2Vec(sentences, size=SEQ_LEN, window=5, min_count=5, workers=4)

pretrained_weights = w2v_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result train embedding shape:', pretrained_weights.shape)

print('\nPreparing train data for modeling...')
x_train = np.zeros([len(sentences), NUM_WORDS], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    if word in w2v_model.wv.vocab:
      x_train[i, t] = word2idx(word)

print('Checking similar words for train model:')
for word in ['friendly', 'cheap', 'delicious', 'chicken', 'pasta']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in w2v_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

# Test data
sentences = [[word for word in txt.lower().split()[:NUM_WORDS]] for txt in test.text.values]
w2v_model = gensim.models.Word2Vec(sentences, size=SEQ_LEN, window=5, min_count=5, workers=4)

pretrained_weights = w2v_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result test embedding shape:', pretrained_weights.shape)

print('\nPreparing test data for modeling...')
x_test = np.zeros([len(sentences), NUM_WORDS], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    if word in w2v_model.wv.vocab:
      x_test[i, t] = word2idx(word)

print('Checking similar words for test model:')
for word in ['friendly', 'cheap', 'delicious', 'chicken', 'pasta']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in w2v_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

y_test = train.drop(labels = ['business_id','text'], axis=1)
y_test = train.drop(labels = ['business_id','text'], axis=1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Result train embedding shape: (21311, 999)

Preparing train data for modeling...
Checking similar words for train model:
  friendly -> courteous (0.77), polite (0.73), personable (0.72), attentive (0.71), efficient (0.67), welcoming (0.67), professional (0.66), cheerful (0.65)
  cheap -> inexpensive (0.72), affordable (0.66), expensive (0.60), reasonable (0.53), pricey (0.49), reliable (0.49), bargain (0.49), healthy (0.49)
  delicious -> tasty (0.83), yummy (0.81), delish (0.80), scrumptious (0.73), amazing (0.72), divine (0.72), phenomenal (0.70), fantastic (0.68)
  chicken -> seitan (0.70), paneer (0.65), oxtail (0.63), gobi (0.60), eggplant (0.59), chana (0.59), goreng (0.58), cauliflower (0.57)
  pasta -> spaghetti (0.71), risotto (0.70), bolognese (0.70), gnocchi (0.69), carbonara (0.68), rigatoni (0.68), linguine (0.67), fettuccine (0.66)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Result test embedding shape: (14171, 999)

Preparing test data for modeling...
Checking similar words for test model:
  friendly -> efficient (0.80), courteous (0.80), attentive (0.79), polite (0.78), prompt (0.73), accommodating (0.73), professional (0.72), helpful (0.71)
  cheap -> affordable (0.68), inexpensive (0.65), expensive (0.58), reasonable (0.56), value (0.55), healthy (0.55), reliable (0.54), priced (0.53)
  delicious -> yummy (0.82), tasty (0.81), delish (0.79), incredible (0.72), amazing (0.70), fantastic (0.68), phenomenal (0.65), divine (0.65)
  chicken -> paneer (0.66), roti (0.64), chix (0.58), oxtail (0.58), cauliflower (0.57), deep (0.56), lamb (0.55), calamari (0.54)
  pasta -> spaghetti (0.75), risotto (0.75), carbonara (0.73), gnocchi (0.71), ravioli (0.65), funghi (0.65), penne (0.65), bolognese (0.64)


# Modeling

## Model 1: One vs. Rest

In [21]:
categories = ['RestaurantsReservations',
       'RestaurantsGoodForGroups', 'OutdoorSeating', 'RestaurantsTakeOut',
       'GoodForKids', 'RestaurantsDelivery', 'BikeParking', 'Wifi']
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

**Processing RestaurantsReservations comments...**
Test accuracy is 0.549645390070922


**Processing RestaurantsGoodForGroups comments...**
Test accuracy is 0.8563829787234043


**Processing OutdoorSeating comments...**
Test accuracy is 0.5709219858156028


**Processing RestaurantsTakeOut comments...**
Test accuracy is 0.9042553191489362


**Processing GoodForKids comments...**
Test accuracy is 0.7039007092198581


**Processing RestaurantsDelivery comments...**
Test accuracy is 0.7304964539007093


**Processing BikeParking comments...**
Test accuracy is 0.8865248226950354


**Processing Wifi comments...**
Test accuracy is 0.5088652482269503




## Model 2: Binary Relevance

In [22]:
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ", accuracy_score(y_test,predictions))

Accuracy =  0.02127659574468085


## Model 3: Multi-label KNN (ML-KNN)

In [0]:
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
lil_x_train = lil_matrix(x_train).toarray()
lil_y_train = lil_matrix(y_train).toarray()
lil_x_test = lil_matrix(x_test).toarray()
# train
classifier_new.fit(lil_x_train, lil_y_train)
# predict
predictions_new = classifier_new.predict(lil_x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))
print("\n")

## Model 4: Neural Network

In [26]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=21311, 
                                    output_dim=100, 
                                    # input_length=seq_len,
                                    dtype='float32'))
# model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.LSTM(units=32))
model.add(tf.keras.layers.LayerNormalization())
# model.add(tf.keras.layers.Dropout(0.1))

# model.add(tf.keras.layers.Conv1D(filters=32, 
#                                  kernel_size=2,
#                                  strides=1,
#                                  activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(units=1024, 
#                                activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=32, 
                               activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(units=y_train.shape[1], 
                               activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2131100   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                17024     
_________________________________________________________________
layer_normalization (LayerNo (None, 32)                64        
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
Total params: 2,149,636
Trainable params: 2,149,572
Non-trainable params: 64
_____________________________________________

In [32]:
type(y_train)

pandas.core.frame.DataFrame

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train, 
          y_train,
          epochs=20,
          batch_size=32)

ValueError: ignored

# Neural Network

Create text to sequence matrices

In [0]:
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300

In [0]:
# num_words = 200000
# seq_len = 30

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(finalDF.text.values)
sequences = tokenizer.texts_to_sequences(finalDF.text.values)
x = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=seq_len)

In [0]:
max_sentence_len = 500
sentences = [[word for word in txt.lower().split()[:max_sentence_len]] for txt in finalDF.text]
w2v_model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=5, workers=4)

In [25]:
pretrained_weights = w2v_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

Result embedding shape: (6660, 200)


In [26]:
print('Checking similar words:')
for word in ['friendly', 'cheap', 'delicious', 'chicken', 'pasta']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in w2v_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

Checking similar words:
  friendly -> attentive (0.94), helpful (0.91), efficient (0.88), accommodating (0.86), polite (0.86), prompt (0.84), welcoming (0.83), pleasant (0.83)
  cheap -> reasonable (0.77), value (0.76), price (0.75), prices (0.74), low (0.72), affordable (0.70), quality (0.69), especially (0.68)
  delicious -> tasty (0.90), amazing (0.87), yummy (0.83), awesome (0.76), incredible (0.76), fantastic (0.76), fresh (0.76), excellent (0.74)
  chicken -> pork (0.87), beef (0.87), roti (0.83), soup (0.83), curry (0.83), poutine (0.82), sandwich (0.81), fried (0.80)
  pasta -> duck (0.90), seafood (0.90), risotto (0.88), lamb (0.88), lobster (0.87), tuna (0.87), curry (0.87), octopus (0.87)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
def word2idx(word):
  return w2v_model.wv.vocab[word].index
def idx2word(idx):
  return w2v_model.wv.index2word[idx]

print('\nPreparing the data for LSTM...')
x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    if word in w2v_model.wv.vocab:
      x[i, t] = word2idx(word)
    
y = np.array(finalDF.iloc[:,2:])

print(x.shape)
print(y.shape)


Preparing the data for LSTM...
(1880, 500)
(1880, 8)


Prepare data

In [29]:
y = y.astype('float32')
x_train, x_tv, y_train, y_tv = train_test_split(x, y, test_size=0.3)
x_valdn, x_test, y_valdn, y_test = train_test_split(x_tv, y_tv, test_size=0.5)

print(x_train.shape, y_train.shape)
print(x_valdn.shape, y_valdn.shape)
print(x_test.shape, y_test.shape)

(1316, 500) (1316, 8)
(282, 500) (282, 8)
(282, 500) (282, 8)


In [0]:
# pathToLogs = '/content/drive/My Drive/Search Project/logs/'
pathToLogs = '/content/logs/'

# Clear any logs from previous runs
!rm -rf {pathToLogs}

Fit model

In [0]:
# Clear session
tf.keras.backend.clear_session()

# Define input

model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Embedding(input_dim=vocab_size,
#                                     output_dim=emdedding_size,
#                                     input_length=200,
#                                     weights=[pretrained_weights],
#                                     dtype='float32'))

lstm_units = 8

model.add(tf.keras.layers.LSTM(units=lstm_units,
                              #  input_shape = (None, max_sentence_len),
                               return_sequences=False))
model.add(tf.keras.layers.LayerNormalization())
# model.add(tf.keras.layers.Dropout(0.5))

# model.add(tf.keras.layers.Reshape((45, 513, 1), input_shape=(45, 513)))
# model.add(tf.keras.layers.Reshape((513, 45, 1), input_shape=(45, 513)))

# model.add(tf.keras.layers.Reshape((45, lstm_units, 1), input_shape=(45, lstm_units)))

# model.add(tf.keras.layers.Conv2D(filters=4,
#                                  kernel_size=(1, 2),
#                                  strides=(1, 2),
#                                  activation='tanh'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPool2D(pool_size=(1,2), strides=(1,2)))

# model.add(tf.keras.layers.Conv1D(filters=32, 
#                                  input_shape=input_shape,
#                                  kernel_size=2,
#                                  strides=1,
#                                  activation='tanh'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPool1D(pool_size=2))

# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(units=8, 
                               activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(units=y_train.shape[1], 
                                activation='sigmoid'))
# model.add(tf.keras.layers.BatchNormalization())

# model.summary()

In [0]:
learning_rate = 0.0002

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
valdn_log_dir = 'logs/gradient_tape/' + current_time + '/valdn'

train_writer = tf.summary.create_file_writer(train_log_dir)
valdn_writer = tf.summary.create_file_writer(valdn_log_dir)


# Sigmoid Cross-entropy with Logits
def computeLoss(labels, logits):
  return tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)


# Exact Match Ratio or Subset Accuracy
def computeAccuracy(truth, predict):
  binaryPred = np.where(predict > 0.5, 1., 0.)
  acc = accuracy_score(y_true=truth, y_pred=binaryPred, normalize=True)
  return acc


def train_step(features, labels, optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)):
  with tf.GradientTape() as tape:
    
    # Compute loss
    logits = model(features, training=True)
    loss = computeLoss(labels, logits)
    loss_value = tf.reduce_mean(loss)
    
    # Compute accuracy
    predictions = model(x_train, training=False)
    acc = computeAccuracy(y_train, predictions)

  # Compute gradients and minimize using the optimizer
  grads = tape.gradient(loss_value, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  return loss_value, acc


def test_step(model, features, labels):
  # Compute loss
  predictions = model(features, training=False)
  loss = computeLoss(labels, predictions)
  loss_value = tf.reduce_mean(loss)
  # Compute accuracy
  acc = computeAccuracy(labels, predictions)

  return loss_value, acc


def train(epochs, batchSize):
  
  # Epoch loop
  for epoch in range(epochs):
    
    # Initialize vars
    epoch_loss = 0
    epoch_acc = 0
    numBatches = math.ceil(x_train.shape[0] / batchSize)
    
    # Batch loop
    for batch in range(numBatches):
      
      # Compute indices
      start = batch*batchSize
      end = (batch+1)*batchSize
      
      # Extract batch
      features, labels = x_train[start:end], y_train[start:end]
      
      # Train
      batch_loss, batch_acc = train_step(features, labels)
      
      # Update local loss and accuracy
      epoch_loss += batch_loss
      epoch_acc += batch_acc
    
    # Update global loss and accuracy
    train_loss = epoch_loss / numBatches
    train_acc = epoch_acc / numBatches

    # Log training loss and accuracy
    with train_writer.as_default():
      tf.summary.scalar('training loss', train_loss, step=epoch)
      tf.summary.scalar('accuracy', train_acc, step=epoch)

    # Validate the model
    valdn_loss, valdn_acc = test_step(model, x_valdn, y_valdn)

    # Log validation loss and accuracy
    with valdn_writer.as_default():
      tf.summary.scalar('training loss', valdn_loss, step=epoch)
      tf.summary.scalar('accuracy', valdn_acc, step=epoch)
    
    # Pretty print output
    template = 'Epoch {}, Training Loss: {}, Training Accuracy: {}, Validation Loss: {}, Validation Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss,
                          train_acc*100,
                          valdn_loss,
                          valdn_acc*100))

In [51]:
train(20, 50)

ValueError: ignored

In [0]:
%tensorboard --logdir logs/gradient_tape
notebook.display(port=6006, height=800)

In [0]:
trainPred = model.predict(x_train)
trainPred[:5]

In [0]:
metrics = model.evaluate(x_test, y_test)
print('test loss, test acc:', metrics)

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/Search Project/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Search Project/model.h5")
print("Saved model to disk")

In [0]:
accuracy_score(np.array([[0, 1, 1],
                         [1, 1, 0],
                         [1, 1, 1],
                         [1, 1, 1],
                         [0, 0, 1]]),
               np.array([[0, 1, 1],
                         [0, 1, 1],
                         [0, 0, 0],
                         [1, 1, 0],
                         [0, 0, 1]]))*100

40.0

# References

[Medium article](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)